## generate waxman training dataset for TELGEN

In [9]:
from solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning
# from scipy.optimize import OptimizeWarning
import warnings
import numpy as np
from functools import partial
import random
import pickle 

In [2]:
root = 'raw/'

### Resource Allocation

#### input: for one graph   
#### G(V, E, c): random graph (strongly connected)
#### (s, t, d) \in [S, T, D]  
#### for every (s, t, d), there is a set p \in Pd (k-shortest path algorithm (4/5/6))

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

### generate and save connected and directed waxman graph different nodes and p
### generate capacities for these graphs

In [4]:
np.random.seed(2024)

def generate_random_capacities(graph):
    for u, v in graph.edges():
        # Generate a random capacity for the edge (u, v)
        capacity = random.uniform(1000, 5000)  # Adjust the range as needed
        # Assign the capacity as an attribute to the edge
        graph[u][v]['capacity'] = capacity
        
        
# generate a directed waxman graph
num_p = [(200, 0.2), (300, 0.18), (400, 0.16), (500, 0.14), (600, 0.12), (700, 0.1), (800, 0.08)] 
# number of nodes and beta value for generating waxman graph

for n in num_p:
    # Generate an waxman random graph
    waxman_graph = nx.waxman_graph(n[0], n[1], seed=2024)

    # Generate capacity for this graph
    generate_random_capacities(waxman_graph)
    Capacity = {}
    for u, v in waxman_graph.edges():
        Capacity[(u, v)] = waxman_graph[u][v]['capacity']
    
    G = waxman_graph.to_directed()
    print('Strongly connected:', nx.is_strongly_connected(G))
    print('# of nodes and edges, beta:', G.number_of_nodes(), G.number_of_edges(), n[1]) 
    
    cc = list(sorted(nx.strongly_connected_components(G), key=len, reverse=True)[:1][0])
    G_sub = G.subgraph(cc)
    print('Strongly connected component:', nx.is_strongly_connected(G_sub))
    print('# of nodes and edges, beta:', G_sub.number_of_nodes(), G_sub.number_of_edges(), n[1]) 
    
    # Clean node and edge attributes to ensure they are GraphML-compatible
    for node, data in G_sub.nodes(data=True):
        for key, value in list(data.items()):
            if not isinstance(value, (str, int, float)):
                data[key] = str(value)
    for u, v, data in G_sub.edges(data=True):
        for key, value in list(data.items()):
            if not isinstance(value, (str, int, float)):
                data[key] = str(value)

    nx.write_graphml(G_sub, root+'waxman_graph/waxman_graph_' + str(n[0]) + 'n_' + str(n[1]) + 'p.graphml')
    
    with open(root+'waxman_graph/Edge_C_' + str(n[0]) + 'n_' + str(n[1]) + 'p.pkl', 'wb') as f:
        pickle.dump(Capacity, f)
    print('Add capacity to one edge first:', len(Capacity.keys()))
    
    print('----------------------------------------')

Strongly connected: False
# of nodes and edges, beta: 200 514 0.2
Strongly connected component: True
# of nodes and edges, beta: 180 504 0.2
Add capacity to one edge first: 257
----------------------------------------
Strongly connected: False
# of nodes and edges, beta: 300 1230 0.18
Strongly connected component: True
# of nodes and edges, beta: 290 1230 0.18
Add capacity to one edge first: 615
----------------------------------------
Strongly connected: False
# of nodes and edges, beta: 400 1844 0.16
Strongly connected component: True
# of nodes and edges, beta: 385 1842 0.16
Add capacity to one edge first: 922
----------------------------------------
Strongly connected: False
# of nodes and edges, beta: 500 2768 0.14
Strongly connected component: True
# of nodes and edges, beta: 499 2768 0.14
Add capacity to one edge first: 1384
----------------------------------------
Strongly connected: False
# of nodes and edges, beta: 600 3170 0.12
Strongly connected component: True
# of nodes a

### generate k-shortest path

In [5]:
from itertools import islice
def k_shortest_paths(G, source, target, k, weight=None):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

### Read all graphs and their capacities and load as a group

In [6]:
#### select graphs ####

num_p = [(200, 0.2), (300, 0.18), (400, 0.16), (500, 0.14), (600, 0.12), (700, 0.1), (800, 0.08)] 

ran_group = []
ran_group_noC = []
for n in num_p:

    G = nx.read_graphml(root+'waxman_graph/waxman_graph_' + str(n[0]) + 'n_' + str(n[1]) + 'p.graphml')
    ran_group_noC.append(G)
    print('Graph info', G.number_of_nodes(), G.number_of_edges())
    print('Connected:', nx.is_strongly_connected(G))
    with open(root+'waxman_graph/Edge_C_' + str(n[0]) + 'n_' + str(n[1]) + 'p.pkl', 'rb') as f:
        Edge_C = pickle.load(f)
    print('Len capacity keys:', len(Edge_C.keys()))
    g_test = nx.DiGraph(G)
    for u, v in G.edges():
        if (int(u), int(v)) in Edge_C.keys():
            g_test.add_edge(u, v, weight=Edge_C[(int(u), int(v))])
        else:
            g_test.add_edge(u, v, weight=Edge_C[(int(v), int(u))])
    ran_group.append(g_test)
    print('After adding Capacity info', g_test.number_of_nodes(), g_test.number_of_edges())
    print('Connected:', nx.is_strongly_connected(g_test))
    print('-------------------------------------')

Graph info 180 504
Connected: True
Len capacity keys: 257
After adding Capacity info 180 504
Connected: True
-------------------------------------
Graph info 290 1230
Connected: True
Len capacity keys: 615
After adding Capacity info 290 1230
Connected: True
-------------------------------------
Graph info 385 1842
Connected: True
Len capacity keys: 922
After adding Capacity info 385 1842
Connected: True
-------------------------------------
Graph info 499 2768
Connected: True
Len capacity keys: 1384
After adding Capacity info 499 2768
Connected: True
-------------------------------------
Graph info 588 3168
Connected: True
Len capacity keys: 1585
After adding Capacity info 588 3168
Connected: True
-------------------------------------
Graph info 693 3904
Connected: True
Len capacity keys: 1952
After adding Capacity info 693 3904
Connected: True
-------------------------------------
Graph info 793 3952
Connected: True
Len capacity keys: 1976
After adding Capacity info 793 3952
Connected

## function define for data generation

In [7]:
# G: G(V, E, C)                           nx.weighted.graph
# STD: demands align with ST pairs        list[([s1, t1], dmd1), ([s2, t2], dmd2),...], (string, int)
# Pd: set of paths for every st pair      dict{[s1, t1]: [([path1], cost1), ([path2], cost2)...], [s2, t2]...}
# # of std pairs = # of keys in Pd
# k: k shortest path for every (s, t, d) tuple

def generate_reallocation(G, STD, Pd, k):
    
    # constraint 1
    A1 = []
    for i in range(len(STD)):
        a = np.zeros(len(STD)*k)
        a[k*i: k*i+k] = 1
        A1.append(a)
    A1 = np.array(A1)
    b1 = np.ones(len(STD))

    # constrain 2
    edges_list = list(G.edges())
    A2 = np.zeros((G.number_of_edges(), len(STD)*k))

    for i in range(len(STD)):
        paths = Pd[tuple(STD[i][0])] # possible paths
        for j in range(k):
            p = paths[j]   # path[j] is the path
            for n in range(len(p)-1):
                if (p[n], p[n+1]) in edges_list:
                    A2[edges_list.index((p[n], p[n+1]))][k*i+j] = STD[i][1]
                else:
                    continue  
    b2 = np.array(list(nx.get_edge_attributes(G,'weight').values()))
    zero_row_indices = np.where(A2.any(axis=1)==0)[0]
    A2 = np.delete(A2, zero_row_indices, axis=0)
    b2 = np.delete(b2, zero_row_indices, axis=0)

    for i in range(A2.shape[0]):
        A2[i] = A2[i]/b2[i]
        b2[i] = b2[i]/b2[i]
    
    # obj
    c = -1*np.concatenate([np.ones(k)*STD[i][1] for i in range(len(STD))])
        
    return A1, b1, A2, b2, c

## train dataset generation

In [8]:
### gen train ####
import time
warnings.filterwarnings("error")

random.seed(2024)
np.random.seed(2024)


pkg_idx = 0              # instance index for your data generation
success_cnt = 0
fail_cnt = 0
bounds = (0., 1.)

max_iter = 15000
num = 1                  # number of instance generated

k = 4                    # k-shortest path
max_d = 5000             # demand max value
min_d = 1000             # demand min value

number_of_st = 10        # number of st pairs

graph_info = []
for g in range(len(ran_group)):
    stds = []
    ips = []
    success_cnt = 0
    times = []
    for n in range(num): # in case failsure case
        
        # generate st pairs with demand value 
        std = []
        Pd = {}
        count_std = 0
        while count_std != number_of_st:
            st = np.random.choice(ran_group[g].nodes(), 2, replace=False)
            d = random.uniform(min_d, max_d)
            k_paths = k_shortest_paths(ran_group_noC[g], st[0], st[1], k=k)
            if len(k_paths) != k:
                continue
            else:
                Pd[(st[0], st[1])] = k_paths
                std.append((st, d))
                count_std += 1
                
        A1, b1, A2, b2, c = generate_reallocation(ran_group[g], std, Pd, k)
        A = np.vstack([A1, A2])
        b = np.hstack([b1, b2])
        
        n_time = time.time()
        try:
            A_eq = None
            b_eq = None
            A_ub = A
            b_ub = b
            res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, 
                          method='interior-point')
            times.append(time.time()-n_time)
            print(res)
        except (LinAlgWarning, OptimizeWarning, AssertionError):
            fail_cnt += 1
            continue
        else:
            if res.success and not np.isnan(res.fun):
                ips.append((torch.from_numpy(A).to(torch.float), torch.from_numpy(b).to(torch.float), torch.from_numpy(c).to(torch.float)))
                success_cnt += 1
                stds.append(std)
        if success_cnt == num:
            break

    with open(root+'raw/instance_'+str(pkg_idx)+'_stds.pkl','wb') as f:
        pickle.dump(stds, f)
    with gzip.open(f'{root}/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
        pickle.dump(ips, file)
    pkg_idx += 1
    graph_info.append((ran_group[g].number_of_nodes(), ran_group[g].number_of_edges(), sum(times)/len(times)))

np.save(root+'/raw/waxman_train_'+str(number_of_st)+'st_info', graph_info)
for i in graph_info:
    print('Graph info and average time used:', i)

    
    
warnings.resetwarnings()

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -21486.501576917006
            x: [ 1.038e-01  5.556e-01 ...  9.056e-02  9.872e-02]
          nit: 10
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -28633.544510214848
            x: [ 2.675e-12  6.939e-01 ...  1.266e-01  2.550e-01]
          nit: 9
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -23926.1409194203
            x: [ 4.090e-01  3.150e-01 ...  2.524e-01  2.785e-01]
          nit: 7
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -22226.910968389726
            x: [ 1.151e-01  1.207e-01 ...  3.450e-01  2.315e-01]
          nit: 9
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          f